# Navigate to rectangle using right palm until intersection is true. Close palm to drag. Open palm to release

In [1]:
import detecthands_multiprocess

In [2]:
import cv2

In [3]:
from multiprocessing import Process


In [4]:
import multiprocessing

In [5]:
import numpy as np

In [6]:
q = multiprocessing.Manager().Queue()
rq=multiprocessing.Manager().Queue()
lq=multiprocessing.Manager().Queue()

In [7]:
t1 = Process(target = detecthands_multiprocess.det_hands, args = (q,rq,lq,))


In [8]:
def show(inq,inrq,inlq):
    recx1=500
    recy1=200
    recx2=550
    recy2=250
    rectangleintersect=False
    prevopen=False
    rectcolor=(255,0,0)
    
    while True:
        data=inq.get()
        rimg=inrq.get()
        limg=inlq.get()
        canvas=np.full_like(rimg, [255,255,255])
        

        
        righthandopen=False
        
        if data['righthandcentre'][0] is not None:

            if data['firstFingerIsOpen']==True and data['secondFingerIsOpen']==True and data['thirdFingerIsOpen']==True and data['fourthFingerIsOpen']==True:
                righthandopen=True
                cv2.putText(canvas,"right hand open", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,255))
                
            if data['firstFingerIsOpen']==False and data['secondFingerIsOpen']==False and data['thirdFingerIsOpen']==False and data['fourthFingerIsOpen']==False:
                righthandopen=False
                cv2.putText(canvas,"right hand closed", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,255))
                
        if data['righthandcentre'][0] is not None:
            canvas = cv2.circle(canvas,(int(data['righthandcentre'][0]*rimg.shape[1]),int(data['righthandcentre'][1]*rimg.shape[0])), 2, (0, 255, 0), 2)
        
        if data['rboundingbox'][0] is not None:
            
            if recx1>data['rboundingbox'][0]*rimg.shape[1] and recx2<data['rboundingbox'][2]*rimg.shape[1] and recy1>data['rboundingbox'][1]*rimg.shape[0] and recy2<data['rboundingbox'][3]*rimg.shape[0]:

                rectangleintersect=True
                rectcolor=(0,255,0)
                
            else:
                if righthandopen==True:
                    rectangleintersect=False
                    rectcolor=(255,0,0)

                
            if righthandopen==False and rectangleintersect==True:
                recx1=int(data['righthandcentre'][0]*rimg.shape[1])
                recy1=int(data['righthandcentre'][1]*rimg.shape[0])
                recx2=recx1+50
                recy2=recy1+50
                if righthandopen==True:
                    rectangleintersect=False
                    rectcolor=(255,0,0)
                    prevopen=True
                
                
        
        canvas=cv2.rectangle(canvas, (recx1,recy1), (recx2,recy2), rectcolor, 2)
        cv2.putText(canvas,str(rectangleintersect), (50,80), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,255))

        



        #print(data)
        
        
        #cv2.putText(image,str(data), (50,170), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,255))
        cv2.imshow('windows', rimg)
        cv2.imshow('windows2', limg)
        cv2.imshow('canvas', canvas)
        if cv2.waitKey(5) & 0xFF == 27:
            break
        
       

In [9]:
t2 = Process(target = show, args = (q,rq,lq,))


In [10]:
t1.start()

In [11]:
t2.start()